<a href="https://colab.research.google.com/github/gabrielfernandorey/ITBA-NLP/blob/main/ITBA_nlp01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo Practico NLP - Detección de Tópicos y clasificación
- ITBA 2024
- Alumno: Gabriel Rey
---

## MODELO

In [12]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
import json
from datetime import datetime, date
from dateutil.parser import parse
from dotenv import load_dotenv

from NLP_tools import Cleaning_text, top_keywords, top_entities, get_topic_name, best_document, clean_all, topic_documents
from core.functions import *

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import unicodedata
from tqdm import tqdm

from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer

In [3]:
from opensearch_data_model import Topic, TopicKeyword, News, os_client, TOPIC_INDEX_NAME, NEWS_INDEX_NAME

In [4]:
from openai import OpenAI

### Inicializamos la base vectorial

In [5]:
init_opensearch()

2024-07-02 21:48:37.061 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`


El índice Topic ya existe. Saltando inicialización de base de datos.
El índice News ya existe. Saltando inicialización de base de datos.


### Path

In [6]:
load_dotenv()
PATH_REMOTO='/content/ITBA-TP/data/'
PATH=os.environ.get('PATH_LOCAL', PATH_REMOTO)
PATH

'C:/Users/gabri/OneDrive/Machine Learning/Github/ITBA-TP/data/'

In [7]:
if PATH == os.environ.get('PATH_LOCAL'):
    client = OpenAI(api_key= os.environ.get('OPENAI_API_KEY'))
else:
    from google.colab import userdata
    client = OpenAI(api_key= userdata.get('OPENAI_API_KEY'))

### Data de noticias original 

In [8]:
df_params = {'0_1000':'0_1000_data.parquet',
             '1000_2000':'1000_2000_data.parquet',
             '2000_3000':'2000_3000_data.parquet',
             'df_joined':'df_joined_2024-04-01 00_00_00.parquet'
            }

chunk = os.environ.get('CHUNK')

df_parquet = pd.read_parquet(PATH+df_params[chunk])
data = list(df_parquet['in__text'])

# Cantidad total de documentos
len(data)

1000

In [ ]:
# Cargar vocabulario
with open(PATH+f'modelos/entities_vocabulary{chunk}.json', 'r') as json_file:
    vocab_entities = json.load(json_file)

### StopWords

In [ ]:
# Stopwords
SPANISH_STOPWORDS = list(pd.read_csv(PATH+'spanish_stop_words.csv' )['stopwords'].values)

### Modelo

In [ ]:
tfidf_vectorizer = TfidfVectorizer(
        tokenizer=None,
        max_df=0.9,
        min_df=0.1,
        ngram_range=(1, 2),
        vocabulary=vocab_entities,
        # max_features=100_000
)
tfidf_vectorizer.fit(data)

Capas del modelo BERTopic

In [ ]:
# Step 1 - Extract embeddings
embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
# Step 2 - Reduce dimensionality
umap_model = UMAP(n_neighbors=15, n_components=10, min_dist=0.0, metric='cosine', random_state=42)
# Step 3 - Cluster reduced embeddings
hdbscan_model = HDBSCAN(min_cluster_size=10, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
# Step 4 - Tokenize topics
vectorizer_model = tfidf_vectorizer
# Step 5 - Create topic representation
ctfidf_model = ClassTfidfTransformer()
# Step 6 - (Optional) Fine-tune topic representations with a `bertopic.representation` model
representation_model = KeyBERTInspired()

# All steps together
topic_model = BERTopic(
  embedding_model=embedding_model,            # Step 1 - Extract embeddings
  umap_model=umap_model,                      # Step 2 - Reduce dimensionality
  hdbscan_model=hdbscan_model,                # Step 3 - Cluster reduced embeddings
  vectorizer_model=vectorizer_model,          # Step 4 - Tokenize topics
  ctfidf_model=ctfidf_model,                  # Step 5 - Extract topic words
  representation_model=representation_model,  # Step 6 - (Optional) Fine-tune topic represenations
  language='spanish',
  verbose=True,
  # calculate_probabilities=True
)

In [ ]:
def my_callback(stage, **kwargs):
    print(f"Stage: {stage}")
    for key, value in kwargs.items():
        print(f"{key}: {value}")

In [ ]:
my_callback(stage="start_training")

### Entrenamiento

In [9]:
# Cargar modelo entrenado o saltar celda y entrenar
ahora = datetime.today()
topic_model = BERTopic.load(PATH+"modelos/bertopic_model_app")
topics = np.load(PATH+"modelos/topics_app.npy")
probs = np.load(PATH+"modelos/probs_app.npy")

In [ ]:
topics, probs = topic_model.fit_transform(data)

In [ ]:
# Grabar modelo
topic_model.save(PATH+"modelos/bertopic_model_nb")
np.save(PATH+"modelos/topics_nb.npy", topics)
np.save(PATH+"modelos/probs_nb.npy", probs)

### Resultados

In [10]:
print(f"Cantidad de tópicos {len(set(topics))} (incluye topico -1)")

Cantidad de tópicos 19 (incluye topico -1)


In [11]:
# Busqueda de documentos por topico, ordenados por mayor probabilidad

T = topic_model.get_document_info(data)
docs_per_topics = T.groupby(["Topic"]).apply(lambda x: x.index).to_dict()

topic =12
#topic = np.random.randint(0, len(docs_per_topics)-1)

print("Ejemplo para tópico:", topic)

doc_probs_x_topic = []
for doc in docs_per_topics[topic]:
    doc_probs_x_topic.append([doc, df_parquet.iloc[doc].in__title, round(probs[doc],4)])

df_query_1 = pd.DataFrame(doc_probs_x_topic)
df_query_1.sort_values(2, ascending=False)

Ejemplo para tópico: 12


,0,1,2
11,407,Advierten que el desguace del Senasa amenaza c...,1.0000
2,65,Sigue la discordia entre los gobernadores y la...,1.0000
3,80,El Congreso constituye mañana la comisión de i...,1.0000
4,108,El Senado activa siete comisiones para salir d...,1.0000
5,124,Un proyecto de ley en Florida busca limitar la...,1.0000
6,130,Los piqueteros preparan nuevas medidas de fuer...,1.0000
14,665,Ley Ómnibus II: el Gobierno llevará el texto f...,1.0000
13,571,Diputados de UCR solicitaron al Gobierno un in...,1.0000
12,410,Despidos: el radicalismo pidió informes por el...,1.0000
10,376,Cobos reclama un informe a Nación por la paral...,1.0000


In [ ]:
df_query_1.iloc[11][1]

### Criterio de corte (umbral)
El criterio de corte utilizado para filtrar las noticias que pertenecen a un topico es el valor de -1 desvio std.

In [ ]:
# Calcular la media, el desvío estándar
mean = np.mean(doc_probs_x_topic)
std_dev = np.std(doc_probs_x_topic)

# Crear el histograma 
plt.hist(doc_probs_x_topic, bins=10, edgecolor='black')

# Añadir líneas para la media, la moda y el desvío estándar
plt.axvline(mean, color='r', linestyle='dashed', linewidth=1, label=f'Media: {mean:.2f}')
plt.axvline(mean - std_dev, color='b', linestyle='dashed', linewidth=1, label=f'-1 STD: {mean - std_dev:.2f}')


# Añadir títulos y etiquetas
plt.title(f'Histograma de probabilidades topico: {topic}')
plt.xlabel('Valor')
plt.ylabel('Frecuencia')
plt.legend()

# Mostrar el gráfico
plt.show()

#### Documentos mas representativos de un topico 

- [1]  Obtenido por el metodo del modelo

In [ ]:
docs_representative = topic_model.get_representative_docs(topic=topic)
docs_representative

- [2] Obtenido por busqueda de probabilidad de documentos perteneciente al topico ( utilizando el umbral de corte )

In [ ]:
# Definir la función de estilo
def color_rows(row, label, value):
    if row[label] >= value:
        return ['color: cyan'] * len(row)
    else:
        return [''] * len(row)
    
# Agrupamos documentos por topico
T = topic_model.get_document_info(data)
docs_per_topics = T.groupby(["Topic"]).apply(lambda x: x.index).to_dict()

# Obtener los IDs de los documentos y sus probabilidades 
docs_ids = []
docs_topic = []
doc_probs_x_topic = []
for doc_ID in tqdm(docs_per_topics[topic]):
    docs_ids.append(df_parquet.index[doc_ID])
    doc_probs_x_topic.append(probs[doc_ID])

# Calcular la media, el desvío estándar
mean = np.mean(doc_probs_x_topic)
std_dev = np.std(doc_probs_x_topic)
threshold = mean - std_dev

# Crear una consulta de múltiples IDs
index_name = 'news'
mget_query = {
    "docs": [{"_index": index_name, "_id": doc_id} for doc_id in docs_ids]
}
# Realizar la búsqueda de múltiples IDs
response = os_client.mget(body=mget_query, index=index_name)

# Procesar la respuesta
for i, doc in enumerate(response['docs']):
    if doc['found']:
        idx = doc['_id']
        title = df_parquet.iloc[docs_per_topics[topic][i]].in__title
        prob_doc = probs[docs_per_topics[topic][i]]
        
        if 'entities' in doc['_source']:
            ent = doc['_source']['entities']
        else:
            ent = []

        docs_topic.append([idx, title, prob_doc, ent])


df_view = pd.DataFrame(docs_topic, columns = ['indice','titulo','prob','entidades']).sort_values('prob', ascending=False)
df_view.style.apply(lambda row: color_rows(row, 'prob', threshold), axis=1)


Nota: Los documentos mas representativos encontrados utilizando el metodo "get_representative_docs" no refleja lo mismo que encontrando los documentos por probabilidades maximas

- [3] Por similitud coseno del topico a los tres documento mas cercanos

In [ ]:
# Cargar los embeddings en un archivo de NumPy o procesar la celda inferior
docs_embedding = np.load(PATH+f"modelos/topic_embeddings_{chunk}.npy")

In [ ]:
# Obtenemos embeddings de todos los documentos
docs_embedding = topic_model.embedding_model.embed(data)

In [ ]:
# Guardar los embeddings en un archivo de NumPy
np.save(PATH+f"modelos/topic_embeddings_{chunk}.npy", docs_embedding)

In [ ]:
# Obtenemos la matriz de similitud coseno entre topicos y documentos
sim_matrix = cosine_similarity(topic_model.topic_embeddings_, docs_embedding)
sim_matrix.shape

In [ ]:
# Similitud coseno entre el topico y los documentos del topico elegido
s_coseno = []
for i in docs_per_topics[topic]:
    s_coseno.append(cosine_similarity([topic_model.topic_embeddings_[topic + 1]], [docs_embedding[i]])[0][0])

# Indices
idx_coseno_sort = np.argsort(s_coseno)[::-1]

for idx in idx_coseno_sort[:3]:
    print(idx, df_parquet.iloc[docs_per_topics[topic][idx]].in__title)

Nota: Del mismo modo que en el punto anterior, los documentos mas cercanos al topico no coinciden no son exactamente los mismos que los hallados en el punto 1

- [4] Primer documento mas cercano al embedding del topico

In [ ]:
# Documento de maxima similitud con el topico

simil_docs_topic = sim_matrix[topic + 1].argmax()
print(f"Noticia de maxima similitud con el topico: {topic}")
print(f"Doc ID: {df_parquet.index[simil_docs_topic]}")
print(f"Titulo: {df_parquet.iloc[simil_docs_topic].in__title}")
print(f"Noticia: {data[simil_docs_topic][:80]}...")
best_doc = data[simil_docs_topic]

""Ojo, con la selección del mas cercano respecto a las probabilidades de los docs del topico""

#### Keywords de solo un topico

In [ ]:
keywords = topic_model.topic_representations_[topic]
topic_keywords = [TopicKeyword(name=keyword, score=score) for keyword, score in keywords if keyword != '']
topic_keywords

- Nota: el umbral de corte para las keywords es la media, se tomaran hasta 10 keywords mientras no supere el umbral de corte.

In [ ]:
freq_k = []
for name_score in topic_keywords:
    freq_k.append(name_score['score'])
umbral_k = np.array(freq_k).mean()
umbral_k

In [ ]:
topic_keywords_top = {}
for name_score in topic_keywords:
    if name_score['score'] >= umbral_k:
        topic_keywords_top[name_score['name']] = name_score['score']

topic_keywords_top


#### Entidades de un Topico a partir de los n documentos mas cercanos al embedding del topico

In [ ]:
# Entidades de documentos ordenados por similitud del topico elelgido
n_docs = 5 # n docs cercanos
entities_topic = []
for doc in df_view[:n_docs].iterrows():
    entities_topic.append(doc[1][3])

from collections import defaultdict 

# Crear un diccionario para contar en cuántos documentos aparece cada palabra
document_frequencies = defaultdict(int)

# Crear un conjunto para cada documento y contar las palabras únicas
for lista in entities_topic:
    unique_words = set(lista)
    for palabra in unique_words:
        document_frequencies[palabra] += 1

# Ordenar las palabras por la frecuencia de documentos de mayor a menor
sorted_frequencies = sorted(document_frequencies.items(), key=lambda item: item[1], reverse=True)

freq_e = []
for item in sorted_frequencies:
    freq_e.append(item[1])
umbral_e = np.array(freq_e).mean()

# Imprimir el resultado ordenado de las primeras 10 entidades segun criterio de corte
topic_entities_top = {}
c=0
for idx in range(len(sorted_frequencies)):
    if sorted_frequencies[idx][1] >= umbral_e:
        if c != 10:
            topic_entities_top[sorted_frequencies[idx][0]] = sorted_frequencies[idx][1]
        else:
            break
        c += 1

topic_entities_top
   

### Grabar todos los registros en Topic y actualizar en News

In [ ]:
# Cargar o saltar carga y procesar celda inferior
with open(PATH+f'modelos/entities{chunk}.json', 'r') as json_file:
    entities = json.load(json_file)

In [ ]:
# Grabar todos los topicos en la base
for topic in topic_model.get_topics().keys():
    if topic > -1:

        topic_keywords_top  = top_keywords(topic, topic_model)
        topic_entities_top  = top_entities(topic, topic_model, docs_embedding, data, entities)
        topic_documents_ids, threshold  = topic_documents(topic, topic_model, probs, df_parquet, data)

        topic_doc = Topic(
            index = topic,   
            name = get_topic_name(''.join({**topic_keywords_top, **topic_entities_top}), client),
            vector = list(topic_model.topic_embeddings_[topic + 1 ]), 
            similarity_threshold = threshold,                      
            created_at = datetime.now(),
            to_date = parse('2024-04-02'),
            from_date = parse('2024-04-01'),         
            keywords = topic_keywords_top,
            entities = topic_entities_top,
            best_doc = best_document(topic, topic_model, docs_embedding, data),
            docs = topic_documents_ids
        ) 

        topic_doc.save()

##### Actualizar datos en News

In [ ]:
# Marcar registros de noticias procesados
index_name = 'news'
search_query = {
    'query': {
        'match': {
            'process': False  
        }
    },
    'size': 10000
}

# Realizar la búsqueda
response = os_client.search( body=search_query, index=index_name )

for i, reg in enumerate(response['hits']['hits']):
    doc_id = reg['_id']
    
    update_body = {
                    "doc": {
                        
                        "process": True
                    }
    }

    # Realizar la actualización
    os_client.update(index=index_name, id=doc_id, body=update_body)

### Recuperar todos los topicos

In [ ]:
index_name = 'topic'

db_topics = []
for i, doc in enumerate(Topic.search().query().scan()):
    db_topics.append(doc.to_dict())
    print(db_topics[i]['index'], db_topics[i]['name'])

### Recuperar documento mas cercano a un topico

In [ ]:
index_name = 'topic'
search_query = {
    'query': {
        'match': {
            'index': 6  # Sustituir 'campo' y 'valor' por campo y valor de búsqueda
        }
    }
}

# Realizar la búsqueda
response = os_client.search(
                            body=search_query,
                            index=index_name
)

texto = response['hits']['hits'][0]['_source']

#Imprimir los resultados
print(f"Topico: {response['hits']['hits'][0]['_source']['name']}")
print("\n"+ texto['best_doc'])

### Nuevo documento

In [ ]:
new_doc = "Bitcoin esta en alza"
new_doc = response['hits']['hits'][0]['_source']['news']
new_doc_embedding = topic_model.embedding_model.embed(new_doc)

# Buscamos en la base a que topico pertenece el nuevo documento
query = {
    "size": 1,
    "query": {
        "knn": {
            "vector": {
                "vector": list(new_doc_embedding),
                "k" : 3
            }
        }
    }
}
response = os_client.search(index='topic', body=query)

print(f"Topico: {response['hits']['hits'][0]['_source']['name']}")
print(f"Estimacion: {response['hits']['hits'][0]['_source']['similarity_threshold']}")

### Busqueda de un documento por su indice, topico asociado, keywords, entities.

In [ ]:

index_name = 'news'
search_query = {
    'query': {
        'match': {
            '_id': '105640350'
        }
    }
}

# Realizar la búsqueda
response = os_client.search(
                            body=search_query,
                            index=index_name
)
print(f"Texto de Noticia: {response['hits']['hits'][0]['_source']['news'][:200]}...\n")

new_doc_embedding = topic_model.embedding_model.embed(response['hits']['hits'][0]['_source']['news'])

# Define el índice y el campo del vector
index_name = 'topic'
vector_field = 'vector'

# Crear una consulta KNN para buscar el embedding más similar
knn_query = {
    "size": 1,  # Número de resultados que deseas obtener, en este caso 1
    "query": {
        "knn": {
            vector_field: {
                "vector": new_doc_embedding,
                "k": 3  # Número de vecinos más cercanos
            }
        }
    }
}

# Realizar la búsqueda
response_topic = os_client.search(index=index_name, body=knn_query)

# Obtener el tópico más cercano
if response_topic['hits']['total']['value'] > 0:
    closest_topic = response_topic['hits']['hits'][0]['_source']
    print(f"El nuevo documento pertenece al tópico: {closest_topic['index']}")
    print(closest_topic['name'])
    print(f"Estimacion: {closest_topic['similarity_threshold']}")
    print(f"Keywords del topico: {closest_topic['keywords']}")
    print(f"Entidades del topico: {closest_topic['entities']}")

else:
    print("No se encontró un tópico cercano.")

### Agrupamiento de topicos

In [ ]:
index_name = 'topic'

db_topics = []
for i, doc in enumerate(Topic.search().query().scan()):
    db_topics.append(doc.to_dict())
    print(db_topics[i]['index'], db_topics[i]['name'])

#### Agrupando por embeddings cercanos

In [ ]:
topic_embeddings = topic_model.topic_embeddings_

In [ ]:
# Reducir la dimensionalidad a 3D usando UMAP
umap_model = UMAP(n_neighbors=5, n_components=3, metric='cosine')
umap_embeddings = umap_model.fit_transform(topic_embeddings)

In [ ]:
import plotly.express as px
import pandas as pd

# Crear un DataFrame para los embeddings y los IDs de los tópicos
df = pd.DataFrame(umap_embeddings, columns=['UMAP Dimension 1', 'UMAP Dimension 2', 'UMAP Dimension 3'])
df['Topic'] = topic_model.get_topic_info()['Topic'].values

# Crear la gráfica 3D interactiva usando Plotly
fig = px.scatter_3d(df, x='UMAP Dimension 1', y='UMAP Dimension 2', z='UMAP Dimension 3',
                    text='Topic', title='Embeddings de los Tópicos Reducidos a 3D con UMAP',
                    width=1000, height=800)  # Ajustar el tamaño del gráfico

# Mostrar la gráfica
fig.show()



In [ ]:
# Eliminamos el topico -1
new_topic_embeddings = topic_embeddings[1:]

In [ ]:
# Calcular la similitud del coseno entre los embeddings de los tópicos
similarities = cosine_similarity(new_topic_embeddings)

In [ ]:
# Crear una matriz de similitud excluyendo la diagonal
np.fill_diagonal(similarities, 0)

In [ ]:
# Encontrar los pares de tópicos más cercanos
topic_pairs = np.dstack(np.unravel_index(np.argsort(similarities.ravel())[::-1], similarities.shape))[0]

In [ ]:
# Mostrar los 5 pares de tópicos más cercanos
for i in range(10):
    topic_id_1, topic_id_2 = topic_pairs[i]
    similarity_score = similarities[topic_id_1, topic_id_2]
    print(f"Topico {topic_id_1} y Topico {topic_id_2} tienen una similitud de: {similarity_score:.4f}")


In [ ]:
topics_to_merge = [16, 17]

id_docs_to_merge = []
for topic in db_topics:
    if topic['index'] in topics_to_merge:
        id_docs_to_merge.append(topic['docs'].keys())

list_id_docs_to_merge = [ item for sublist in id_docs_to_merge for item in sublist ]


In [ ]:
index_name = 'news'

# Construir el cuerpo de la solicitud para `mget`
body = {
    "docs": [{"_index": "news", "_id": int(doc_id)} for doc_id in list_id_docs_to_merge]
}

# Realizar la solicitud `mget`
response = os_client.mget(body=body)


In [ ]:
idx_relativo = []
docs_input = []
for i, doc in enumerate(response['docs']):
    idx_relativo.append(doc['_id'])
    docs_input.append(doc['_source']['news'])

    

In [ ]:
len(docs_input)

In [ ]:
topic_model.merge_topics(docs_input, topics_to_merge)

In [13]:
# Configurar la busqueda de todas las noticias no procesadas ( False ) en la base (al menos 10.000)
index_name = 'news'
search_query = {
    'query': {
        'match': {
            'process': True
        }
    },
    'size': 10000
}

# Realizar la búsqueda
response = os_client.search( body=search_query, index=index_name )

db_news = []
for reg in response['hits']['hits']:
    _id =  reg['_id']
    title =  reg['_source']['title']
    news =  reg['_source']['news']
    try:
        keywords =  reg['_source']['keywords'] 
    except:
        keywords = ['']
    try:
        entities =  reg['_source']['entities'] 
    except:
        entities = ['']
    created_at =  reg['_source']['created_at'] 
    
    db_news.append([_id, title, news, keywords, entities, created_at])

df_news = pd.DataFrame(db_news , columns=["indice", "titulo", "noticia", "keywords", "entidades", "creado"])

In [19]:
topic=10

In [21]:
T = topic_model.get_document_info(data)
docs_per_topics = T.groupby(["Topic"]).apply(lambda x: x.index).to_dict()

# Obtener los IDs de los documentos y sus probabilidades 
docs_IDs = {}
doc_probs_x_topic = []
for doc_idx in docs_per_topics[topic]:
    
    docs_IDs[df_news.indice[doc_idx]] = probs[doc_idx]
    doc_probs_x_topic.append(probs[doc_idx])

# Calcular la media, el desvío estándar
mean = np.mean(doc_probs_x_topic)
std_dev = np.std(doc_probs_x_topic)
threshold = mean - std_dev

# Filtra los docs que superan o igualan al valor del umbral calculado
filter = {}
for k,v in docs_IDs.items():
    if v >= threshold:
        filter[k] = v

In [23]:
# Ordeno de mayor a menor
ids_filter_sort = dict(sorted(filter.items(), key=lambda item: item[1], reverse=True))

In [26]:
ids_filter_sort

{'105638222': 1.0,
 '105628246': 1.0,
 '105641111': 1.0,
 '105595584': 1.0,
 '105579195': 1.0,
 '105593527': 1.0,
 '105580036': 1.0,
 '105558763': 1.0,
 '105595323': 1.0,
 '105579437': 1.0,
 '105593407': 0.9591836188380196,
 '105594298': 0.9536858051294697,
 '105595318': 0.8797894030692698,
 '105579944': 0.8765740589839246,
 '105650270': 0.8544252505331561,
 '105595196': 0.6497119700491902}

In [50]:
title_filter_sort = [ df_news.loc[df_news['indice'] == idx].values[0][1] for idx in ids_filter_sort.keys() ]



In [51]:
title_filter_sort 

['El régimen de Nicolás Maduro excarceló al youtuber acusado de terrorismo en Venezuela',
 'Autismo en adultos: la importancia de derribar los estigmas para mejorar su entendimiento',
 'La lectura en voz alta: clave para el desarrollo infantil en el hogar y la escuela',
 'Dieron de alta a Roxy Vázquez tras ser internada: "Fue un cuadro viral severo"',
 'Internaron nuevamente a Jorge Lanata',
 'Sube la nafta más del 4 por ciento: cuánto cuesta el combustible desde abril',
 'Subte: reabrió la estación Facultad de Medicina de la Línea D',
 'Atentado en Israel: un terrorista palestino acuchilló a un niño y fue abatido por la policía - MDZ Online',
 'Bombardeo de Israel a la embajada de Irán en Siria mata a 8 personas',
 'Israel bombardea la capital de Siria: hay seis muertos',
 'Más aumentos: el Gobierno desreguló el precio del pan y la yerba',
 "Podemos recurre al TC para que el Supremo investigue su querella contra el juez García Castellón por supuesto 'lawfare'",
 'Dos generales de la G

In [52]:
' '.join(title_filter_sort)

'El régimen de Nicolás Maduro excarceló al youtuber acusado de terrorismo en Venezuela Autismo en adultos: la importancia de derribar los estigmas para mejorar su entendimiento La lectura en voz alta: clave para el desarrollo infantil en el hogar y la escuela Dieron de alta a Roxy Vázquez tras ser internada: "Fue un cuadro viral severo" Internaron nuevamente a Jorge Lanata Sube la nafta más del 4 por ciento: cuánto cuesta el combustible desde abril Subte: reabrió la estación Facultad de Medicina de la Línea D Atentado en Israel: un terrorista palestino acuchilló a un niño y fue abatido por la policía - MDZ Online Bombardeo de Israel a la embajada de Irán en Siria mata a 8 personas Israel bombardea la capital de Siria: hay seis muertos Más aumentos: el Gobierno desreguló el precio del pan y la yerba Podemos recurre al TC para que el Supremo investigue su querella contra el juez García Castellón por supuesto \'lawfare\' Dos generales de la Guardia Revolucionaria y otras 6 personas murier

In [42]:
df_news

,indice,titulo,noticia,keywords,entidades,creado
0,105638886,Paso a paso para invertir en criptomonedas y c...,Las criptomonedas se caracterizan por usar red...,"[moneda, monedas virtuales, bitcoin, criptomon...","[Tether US, Elon Musk, Rodrigo Sura, Nayib Buk...",2024-07-02T20:22:51.549910
1,105628162,"Petro calificó de ""golpe antidemocrático"" la i...",Declaraciones de Gustavo Petro durante la inve...,"[derecho, comunidad internacional, elecciones,...","[Gustavo Petro, Leopoldo López, Corina Yoris, ...",2024-07-02T20:22:51.552599
2,105638375,"Por primera vez, Nicole Neumann reveló los mot...",Años felices en familia: Nicole Neumann junto ...,"[hijas, momento, vida, modelo, hija, días]","[Nicole Neumann, Fabián Cubero, Indiana Cubero...",2024-07-02T20:22:51.552599
3,105638272,Cómo es la fábrica de drones Shahed donde Ucra...,Un ataque ucraniano con drones en la región ce...,"[producción, aviones, planta rusa, planta, alc...","[Washington Post, Roman Petushkov, John Hardie...",2024-07-02T20:22:51.553161
4,105628203,Un proyecto de ley en Florida busca limitar la...,La iniciativa legislativa en Florida enfrenta ...,"[orientación sexual, proyecto, banderas, espac...","[NBC Miami, CBS Miami, David Borrero, Randy Fi...",2024-07-02T20:22:51.553161
...,...,...,...,...,...,...
995,105580240,Weretilneck no viaja a la vigilia de Malvinas ...,"El gobernador de Río Negro, Alberto Weretilnec...","[coyuntura provincial, fecha, semana, gobernad...","[Río Negro, Alberto Weretilneck, Gustavo Melel...",2024-07-02T20:22:51.795240
996,105558638,Conflicto con Colombia: El regreso del embajador,Con un comunicado conjunto entre las canciller...,"[comunicado, relaciones, países, relaciones di...","[Javier Milei, Gustavo Petro, Gobierno colombi...",2024-07-02T20:22:51.795240
997,105593542,Sorpresa en el INDEC: por qué ofertas de super...,Luis Caputo les pide a las grandes cadenas que...,"[productos, grandes cadenas, cadenas, precios,...",[Luis Caputo],2024-07-02T20:22:51.795240
998,105592124,"Tras la desregulación del precio de la yerba, ...",El Instituto Nacional de la Yerba Mate marcó q...,"[valores, yerba, hoja verde, hoja, libre merca...","[Javier Milei, Marcelo Hacklander]",2024-07-02T20:22:51.795240


In [17]:

x_ids, x_titles, umbral = topic_documents(topic, topic_model, probs, df_news, data)

In [15]:
x_titles

{}

In [16]:
x_ids

{}